<center>
    <p style="text-align:center">
        <img alt="arize llama-index logos" src="https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/arize_llamaindex.png" width="400">
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing the execution of your LlamaIndex application</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LlamaIndex. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Parts of this notebook require an [OpenAI API key](https://platform.openai.com/account/api-keys) to run. If you don't have an OpenAI key, you can still run Phoenix by skipping cells preceded by the 💭 emoji.


## Chatbot Architecture

Your chatbot was built using LlamaIndex's low-level API. The architecture of your chatbot is shown below and can be explained in five steps.

![llama-index chatbot architecture](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/llama_index_chatbot_architecture.png)

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to the OpenAI embeddings API to embed the user query using the `text-embedding-ada-002` model.
1. `llama_index.retrievers.RetrieverQueryEngine` does a similarity search against the entries of your index knowledge base for the two most similar pieces of context by cosine similarity.
1. `llama_index.indices.query.ResponseSynthesizer` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI chat completions API with the `gpt-3.5-turbo`.
2. The response is returned to the user.

## 1. Install Dependencies and Import Libraries

Install LlamaIndex and other dependencies.

In [ ]:
!pip install -q gcsfs llama-index

Import libraries.

In [ ]:
import os
import textwrap
from datetime import timedelta

import numpy as np
import openai
import pandas as pd
import phoenix as px
from phoenix.experimental.callbacks.llama_index_trace_callback_handler import (
    OpenInferenceTraceCallbackHandler,
)
from gcsfs import GCSFileSystem
from IPython.display import YouTubeVideo
from langchain.chat_models import ChatOpenAI
from llama_index import LLMPredictor, ServiceContext, StorageContext, load_index_from_storage
from llama_index.callbacks import CallbackManager
from llama_index.callbacks.open_inference_callback import as_dataframe
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore


pd.set_option("display.max_colwidth", 1000)

## 2. Configure Your OpenAI API Key

In [ ]:
import getpass

if os.environ["OPENAI_API_KEY"] is None:
    openai_api_key = getpass.getpass("🔑 Enter your OpenAI API key: ")
    openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai.api_key

## 3. Download Your Knowledge Base

Download your pre-built index from cloud storage and instantiate your storage context.

In [ ]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)

Download and unzip a pre-built knowledge base index consisting of chunks of the Arize documentation.

## 4. Run Your Question-Answering Service

💭 Start a LlamaIndex application from your downloaded index. Use the `OpenInferenceCallbackHandler` to store your data in [OpenInference format](https://github.com/Arize-ai/open-inference-spec), an open standard for capturing and storing AI model inferences that enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Arize and Phoenix.

In [ ]:
callback_handler = OpenInferenceTraceCallbackHandler()
service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()

💭 Ask a few questions of your question-answering service and view the responses.

In [ ]:
max_line_length = 80
for query in [
    "Why is arize so expensive?",
    "Is my model doing okay?",
    "How do I need to format timestamps?",
    "What is the price of the Arize platform",
]:
    print("Query")
    print("=====")
    print()
    print(textwrap.fill(query, max_line_length))
    print()
    response = query_engine.query(query)
    print("Response")
    print("========")
    print()
    print(textwrap.fill(str(response), max_line_length))
    print()

In [ ]:
from phoenix.trace.span_json_encoder import spans_to_jsonl
from phoenix.trace.trace_dataset import TraceDataset
from phoenix.trace.utils import json_lines_to_df
from phoenix import TraceDataset

ds = TraceDataset.from_spans(callback_handler._tracer.span_buffer)

px.launch_app(trace=ds)

In [ ]:
(
    ds.dataframe["end_time"].astype("datetime64[ms]")
    - ds.dataframe["start_time"].astype("datetime64[ms]")
).dt.total_seconds() * 1000